In [1]:
import os
import sys
import tqdm
import pandas as pd

In [2]:
REBADD_LIB_PATH = os.path.abspath(os.pardir)
if REBADD_LIB_PATH not in sys.path:
    sys.path = [REBADD_LIB_PATH] + sys.path

from rebadd.evaluate import evaluate_sr_nov_div

In [3]:
## Test data (navitoclax, ABT-737)
filepath_ref = os.path.join(os.pardir, 'data', 'zinc15', 'zinc15_test.txt')
df_ref = pd.read_csv(filepath_ref, header=None)

referece_smiles_iter = df_ref.iloc[:,0].values.tolist()

print(len(referece_smiles_iter))

2


In [4]:
input_dir = 'outputs_7_calculate_properties_generated'

modelnames = [
    'char+reinforce+scst+offpolicy',
]

filenames = [f'smi_after.csv.{num}' for num in range(10)]

In [5]:
frames = []

for modelname in modelnames:
    
    for filename in filenames:
        
        filepath = os.path.join(input_dir, modelname, filename)
        
        df = pd.read_csv(filepath)
        
        df = df.loc[:,('smiles', 'bcl2', 'bclxl', 'bclw')]
        df.loc[:,'model'] = modelname
        df.loc[:,'checkpoint'] = filename.split('.')[-1]
        
        frames.append(df)

In [6]:
data = []

for df in tqdm.tqdm(frames):

    s_sr, s_nov, s_div = evaluate_sr_nov_div(df, referece_smiles_iter, 'bcl2_bclxl_bclw')
    
    data.append({'SR':s_sr, 'Nov':s_nov, 'Div':s_div, 'Model':df.loc[0,'model'], 'Ckpt':df.loc[0,'checkpoint']})

100%|██████████| 10/10 [00:05<00:00,  2.00it/s]


In [7]:
df_records = pd.DataFrame(data)
df_records.loc[:,'HMean'] = (df_records.loc[:,'SR'] * df_records.loc[:,'Nov'] * df_records.loc[:,'Div']) ** 0.333

df_records

,SR,Nov,Div,Model,Ckpt,HMean
0,0.9728,1.0,0.525842,char+reinforce+scst+offpolicy,0,0.799938
1,0.9746,1.0,0.511101,char+reinforce+scst+offpolicy,1,0.792888
2,0.9740,1.0,0.517663,char+reinforce+scst+offpolicy,2,0.796100
3,0.9708,1.0,0.541377,char+reinforce+scst+offpolicy,3,0.807179
4,0.9740,1.0,0.511974,char+reinforce+scst+offpolicy,4,0.793176
5,0.9696,1.0,0.528480,char+reinforce+scst+offpolicy,5,0.800394
6,0.9702,1.0,0.539893,char+reinforce+scst+offpolicy,6,0.806275
7,0.9744,1.0,0.531939,char+reinforce+scst+offpolicy,7,0.803455
8,0.9746,1.0,0.541425,char+reinforce+scst+offpolicy,8,0.808253
9,0.9718,1.0,0.533536,char+reinforce+scst+offpolicy,9,0.803542


In [8]:
df_records.groupby('Model').mean(numeric_only=True)

,SR,Nov,Div,HMean
Model,,,,
char+reinforce+scst+offpolicy,0.97268,1.0,0.528323,0.80112


In [9]:
df_records.groupby('Model').std(numeric_only=True)

,SR,Nov,Div,HMean
Model,,,,
char+reinforce+scst+offpolicy,0.001937,0.0,0.011546,0.005612
